In [ ]:
!pip install -U git+https://github.com/aguirrejuan/ConvRFF.git --quiet

In [ ]:
import wandb
import numpy as np 
import os 
from glob import glob
import tensorflow as tf 
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import pandas as pd 

from convRFF.class_activation_maps.cam_data import mimic_mmap
from convRFF.class_activation_maps import load_model_from_run
from convRFF.class_activation_maps import generator_output_model
from convRFF.class_activation_maps import save_ouput_model_data
from convRFF.class_activation_maps import get_path_run
from convRFF.class_activation_maps.output_masked import DTYPE as dtype_out_masked

from gcpds.image_segmentation.datasets.segmentation import InfraredThermalFeet
from gcpds.image_segmentation.losses import DiceCoefficient

import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

WANDB_PROJECT = 'thesis_experiments'
ROOT_FOLDER = '/content/drive/MyDrive/Pruebas_Tesis/cam_data_refactor_database' #ROOT_FOLDER_TO_SAVE_CAMS
api = wandb.Api()
runs = api.runs(WANDB_PROJECT)

In [ ]:
def get_dataset(name_dataset):
    if 'test' in name_dataset:
        name_dataset = '_'.join(name_dataset.split('_')[1::])
    datasets = {
                'infrared_thermal_feet':InfraredThermalFeet(split=[0.1,0.1],seed=42),
                'infrared_thermal_feet_nodataaug':InfraredThermalFeet(split=[0.1,0.1],seed=42)}
    return datasets[name_dataset]


def get_path(dataset, id_run):
    name_file = f'colab11_thesis_experiments_{id_run}_*/{dataset}_layercam.mimic_memmap'
    file_path = os.path.join(ROOT_FOLDER, name_file)
    file_path = glob(file_path)
    if len(file_path) ==0:
        return None 
    return file_path[0] 



def calculate_results(arr_mmap, layer, dataset, target_classes=[0,1]):
    data = arr_mmap[layer]
    len_ = len(data['info_instance'])
    results = np.zeros((len_, 3, len(target_classes)))
    type_nerves = []
    for i, (img, mask_, *info) in enumerate(dataset):
        *label, id_image = info
        #print(id_image)
        filter_ = np.argmax(data['info_instance'][:,1] == id_image)

        name_nerve, id_img = data['info_instance'][filter_]
        cam = data['cam'][filter_]

        #img, mask_, label, id_image  = dataset.load_instance_by_id(id_img)
        img = tf.image.resize(img, cam.shape[:2])
        mask_ = tf.image.resize(mask_, cam.shape[:2])

    
        for target_class in target_classes:
            #p,p, w,w, d
            if len(target_classes) == 2:
                mask = mask_ if target_class==1 else 1 - mask_
            else:
                mask = mask_[...,target_class][...,None]

            cam_target = cam[...,target_class][...,None]

            total_e = tf.reduce_sum(cam_target) + 1e-9
            partial_e = tf.reduce_sum(mask*cam_target)/total_e
            results[i, 0, target_class] = partial_e

            N = tf.reduce_sum(mask)
            vnorm = tf.reduce_sum(mask*cam_target)/N
            results[i, 1, target_class] = vnorm

            norm_cam = cam_target/(tf.reduce_max(cam_target)+1e-9)
            dice = DiceCoefficient()(mask,norm_cam)
            results[i,2,target_class] = dice 
            

        type_nerves.append(name_nerve)
        #results[:,1,:] = results[:,1,:]/(results[:,1,:].max(axis=1,keepdims=True)+1e-9)
    return results, type_nerves




def get_results_layers(arr_mmap, dataset, target_class):
    layers = arr_mmap._keys.keys()
    #layers = ['Res15_Conv02','Res17_Conv02']
    results = {}
    for layer in tqdm(layers):
        r, type_nerves = calculate_results(arr_mmap, layer, dataset,target_class)
        results.setdefault('layer',[]).extend([layer]*len(type_nerves))
        #p (c1,c2,c3,...), w(c1,c2,c3,...), d
        for i, metric in enumerate(['p','w','dice']):
            for class_ in range(r.shape[2]):
                results.setdefault(f'{metric}_{class_}',[]).extend(r[:,i,class_])                
        results.setdefault('class',[]).extend(type_nerves)
    return results 


def load_mimic_mmap(file_path, dtype):
    return mimic_mmap(file_path, dtype=dtype, mode='r')

def generate_data_frame_from_ids(ids, dataset, dtype,target_class, dataset_index=0):
    dataset_obj = get_dataset(dataset)()[dataset_index]
    for id_run in ids: 
        file_path = get_path(dataset, id_run)
        if not file_path:
            print(f'Id dosent exist: {id_run}')
            continue 
        df_name = os.path.dirname(file_path)
        df_name = os.path.join(df_name,f'{dataset}.csv')
        name_model = '_'.join(os.path.split(os.path.dirname(file_path))[-1].split('_')[4::])
        print(name_model, id_run)
        if  not os.path.exists(df_name):
            try:
                arr_mmap = load_mimic_mmap(file_path, id_run)
                results = get_results_layers(arr_mmap, dataset_obj,target_class)
                results = pd.DataFrame(results)
                results['id'] = id_run
                results['model'] = name_model
                results.to_csv(df_name, index=False)
            except: 
                print(f'error code {id_run}')

In [ ]:
ids1 = [run.id for run in runs if  run.state=='finished' and run.config['dataset'] == 'infrared_thermal_feet' and run.sweep.id == '6l60ok26']

ids2 = [run.id for run in runs if  run.state=='finished' and run.config['dataset'] == 'infrared_thermal_feet_nodataaug' and run.sweep.id == 'hg11kida']


ids = ids1 + ids2

dataset = 'infrared_thermal_feet'
DTYPE = np.dtype([('info_instance', 'U50', (121,2)),  
                  ('cam', np.float32, (121,224, 224, 2))
                 ])

In [ ]:
generate_data_frame_from_ids(ids, dataset, DTYPE, target_class=[0,1], dataset_index=0)